In [1]:
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn import model_selection
from sklearn import metrics
from skopt import dump, load
from skopt.plots import plot_convergence
from scipy.stats import reciprocal
import skopt
from skopt import gp_minimize
import numpy as np
import pandas as pd
import keras_tuner as kt
from sklearn import metrics
import datetime
import re
import os
import time
import seaborn as sns
import matplotlib.pyplot as plt

# Reusable Constants

In [2]:
CALLBACKS = [keras.callbacks.EarlyStopping(patience=5, monitor='val_mean_squared_error', restore_best_weights=True)]

# Load Data from Sources

In [3]:
df = pd.read_excel('etching.xlsx')

In [4]:
df.columns

Index(['Pressure', 'RF Power', 'Electrode Gap', 'CCL_4 Flow', 'HE Flow',
       'O2 Flow', 'Etch Rate -Rp A/min', 'Etch Uniformity',
       'Oxide Selectivity - Sox', 'Photoresist Selectivity - Sph'],
      dtype='object')

In [5]:
df.shape

(53, 10)

# Data Pre-Processing

In [6]:
x = df[['Pressure', 'RF Power', 'Electrode Gap', 'CCL_4 Flow', 'HE Flow', 'O2 Flow']].to_numpy()
y = df[['Etch Rate -Rp A/min', 'Etch Uniformity', 'Oxide Selectivity - Sox', 'Photoresist Selectivity - Sph']].to_numpy()

In [7]:
x_scaler = MinMaxScaler().fit(x)
x_scaled = x_scaler.transform(x)
y_scaler = MinMaxScaler().fit(y)
y_scaled = y_scaler.transform(y)

In [10]:
y_test_etch_rate = [ x[0] for x in y_scaled ] 
y_test_etch_uni = [ x[1] for x in y_scaled ] 
y_test_ox_sel = [ x[2] for x in y_scaled ] 
y_test_pho_sel = [ x[3] for x in y_scaled ] 

# Multi-Model Test

In [11]:
x_scaled = x_scaled
y_etch_rate = np.array(y_test_etch_rate)
y_etch_uni = np.array(y_test_etch_uni)
y_ox_sel = np.array(y_test_ox_sel)
y_ph_sel = np.array(y_test_pho_sel)

In [12]:
MULTI_SPACE = [
    skopt.space.Integer(5, 12, name='n_units_one'),
    skopt.space.Real(0.0, .5, name='dropout_rate'),
    skopt.space.Categorical(['relu', 'selu'], name='activation'),
    skopt.space.Categorical(['sigmoid', None], name='output_activation'),
    skopt.space.Real(1e-4, .1, prior='log-uniform', name='learning_rate'),
    skopt.space.Categorical(['l1', 'l2', 'l1_l2', None], name='regularize')
]

In [13]:
def build_multi_model(n_units=10, activation='relu', output_activation=None, learning_rate=1e-3, dropout_rate=0, optimizer='adam', regularize=None):
    model = keras.models.Sequential()
    
    # input layer
    model.add(keras.layers.InputLayer(input_shape=(6,), name='input_layer'))
    
    # hidden layers
    model.add(keras.layers.Dense(
        units=n_units, 
        activation=activation, 
        name=f'hidden_layer',
        kernel_regularizer=regularize,
        bias_regularizer=regularize
    ))
    
    model.add(keras.layers.Dropout(rate=dropout_rate, name='dropout'))
        
    # output layer
    model.add(keras.layers.Dense(1, activation=output_activation, name='output_layer'))
    
    model.compile(
        loss='mean_squared_error',
        optimizer=optimizer,
        learning_rate=learning_rate,
        metrics=['mean_squared_error']
    )
    
    return model

In [34]:
@skopt.utils.use_named_args(MULTI_SPACE)
def objective_multi_er(**params):  
    valid_mse_scores = []
    cv = LeaveOneOut()
    for train_idx, test_idx in cv.split(x_scaled):
        # Get train valid splits
        x_train, x_valid = x_scaled[train_idx], x_scaled[test_idx]
        y_train, y_valid = y_etch_rate[train_idx], y_etch_rate[test_idx]
        # fit model
        model = build_multi_model(
            n_units=params['n_units_one'],
            learning_rate=params['learning_rate'],
            activation=params['activation'],
            output_activation=params['output_activation'],
            dropout_rate=params['dropout_rate'],
            regularize=params['regularize']
        )
        model.fit(x_train, y_train, epochs=50, validation_data=(x_valid, y_valid), verbose=0)
        trial_score = model.evaluate(x_valid, y_valid, verbose=0)[1]
        valid_mse_scores.append(trial_score)
    print(f'Avg Validation MSE: {np.mean(valid_mse_scores)}')
    return np.mean(valid_mse_scores)

In [33]:
@skopt.utils.use_named_args(MULTI_SPACE)
def objective_multi_eu(**params):
    valid_mse_scores = []
    cv = LeaveOneOut()
    for train_idx, test_idx in cv.split(x_scaled):
        # Get train valid splits
        x_train, x_valid = x_scaled[train_idx], x_scaled[test_idx]
        y_train, y_valid = y_etch_uni[train_idx], y_etch_uni[test_idx]
        # fit model
        model = build_multi_model(
            n_units=params['n_units_one'],
            learning_rate=params['learning_rate'],
            activation=params['activation'],
            output_activation=params['output_activation'],
            dropout_rate=params['dropout_rate'],
            regularize=params['regularize']
        )
        model.fit(x_train, y_train, epochs=50, validation_data=(x_valid, y_valid), verbose=0)
        trial_score = model.evaluate(x_valid, y_valid, verbose=0)[1]
        valid_mse_scores.append(trial_score)
    print(f'Avg Validation MSE: {np.mean(valid_mse_scores)}')
    return np.mean(valid_mse_scores)

In [32]:
@skopt.utils.use_named_args(MULTI_SPACE)
def objective_multi_os(**params):
    valid_mse_scores = []
    cv = LeaveOneOut()
    for train_idx, test_idx in cv.split(x_scaled):
        # Get train valid splits
        x_train, x_valid = x_scaled[train_idx], x_scaled[test_idx]
        y_train, y_valid = y_ox_sel[train_idx], y_ox_sel[test_idx]
        # fit model
        model = build_multi_model(
            n_units=params['n_units_one'],
            learning_rate=params['learning_rate'],
            activation=params['activation'],
            output_activation=params['output_activation'],
            dropout_rate=params['dropout_rate'],
            regularize=params['regularize']
        )
        model.fit(x_train, y_train, epochs=50, validation_data=(x_valid, y_valid), verbose=0)
        trial_score = model.evaluate(x_valid, y_valid, verbose=0)[1]
        valid_mse_scores.append(trial_score)
    print(f'Avg Validation MSE: {np.mean(valid_mse_scores)}')
    return np.mean(valid_mse_scores)

In [31]:
@skopt.utils.use_named_args(MULTI_SPACE)
def objective_multi_ps(**params):
    valid_mse_scores = []
    cv = LeaveOneOut()
    for train_idx, test_idx in cv.split(x_scaled):
        # Get train valid splits
        x_train, x_valid = x_scaled[train_idx], x_scaled[test_idx]
        y_train, y_valid = y_ph_sel[train_idx], y_ph_sel[test_idx]
        # fit model
        model = build_multi_model(
            n_units=params['n_units_one'],
            learning_rate=params['learning_rate'],
            activation=params['activation'],
            output_activation=params['output_activation'],
            dropout_rate=params['dropout_rate'],
            regularize=params['regularize']
        )
        model.fit(x_train, y_train, epochs=50, validation_data=(x_valid, y_valid), verbose=0)
        trial_score = model.evaluate(x_valid, y_valid, verbose=0)[1]
        valid_mse_scores.append(trial_score)
    print(f'Avg Validation MSE: {np.mean(valid_mse_scores)}')
    return np.mean(valid_mse_scores)

In [35]:
er_opt_results = gp_minimize(objective_multi_er, MULTI_SPACE, n_calls=20, random_state=0)
dump(er_opt_results, 'tuning/er_opt_results')

Avg Validation MSE: 0.15874597430229187
Avg Validation MSE: 0.05742276459932327
Avg Validation MSE: 0.16255216300487518
Avg Validation MSE: 0.1857496052980423
Avg Validation MSE: 0.08427127450704575
Avg Validation MSE: 0.06334074586629868
Avg Validation MSE: 0.0509733147919178
Avg Validation MSE: 0.062471818178892136
Avg Validation MSE: 0.1007326990365982
Avg Validation MSE: 0.06493272632360458
Avg Validation MSE: 0.05032914876937866
Avg Validation MSE: 0.0652846172451973
Avg Validation MSE: 0.05361078307032585
Avg Validation MSE: 0.057985350489616394
Avg Validation MSE: 0.06350110471248627


KeyboardInterrupt: 

In [ ]:
eu_opt_results = gp_minimize(objective_multi_eu, MULTI_SPACE, n_calls=20, random_state=0)
dump(eu_opt_results, 'tuning/eu_opt_results')

In [ ]:
os_opt_results = gp_minimize(objective_multi_os, MULTI_SPACE, n_calls=20, random_state=0)
dump(os_opt_results, 'tuning/os_opt_results')

In [ ]:
ps_opt_results = gp_minimize(objective_multi_ps, MULTI_SPACE, n_calls=20, random_state=0)
dump(ps_opt_results, 'tuning/ps_opt_results')